<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/llama13b_japancese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece accelerate bitsandbytes
!pip install git+https://github.com/huggingface/peft.git

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel
from transformers import GenerationConfig


device = "cuda" if torch.cuda.is_available() else "cpu"

base_model = "decapoda-research/llama-13b-hf"
# Please note that the special license of decapoda-research/llama-13b-hf is applied.
# model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16, device_map={'': 0},)
model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16, device_map="auto")

tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = PeftModel.from_pretrained(
    model,
    "izumi-lab/llama-13b-japanese-lora-v0",
    torch_dtype=torch.float16,
    # device_map={'': 0},
    device_map="auto",
)

In [2]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""


model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        **kwargs,
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

In [3]:
instruction = "最新のAIについて教えてほしい。"
response = evaluate(instruction)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│ in evaluate:44                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:743 in generate                       │
│                                                                                                  │
│    740 │   │   │   self.base_model.generation_config = self.generation_config                    │
│    741 │   │   try:                                                                              │
│    742 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  743 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    744 │   │   │   else:                                                                         │
│    745 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    746 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1604 in generate        │
│                                                                                                  │
│   1601 │   │   │   │   **model_kwargs,                                                           │
│   1602 │   │   │   )                                                                             │
│   1603 │   │   │   # 13. run beam search                                                         │
│ ❱ 1604 │   │   │   return self.beam_search(                                                      │
│   1605 │   │   │   │   input_ids,                                                                │
│   1606 │   │   │   │   beam_scorer,                                                              │
│   1607 │   │   │   │   logits_processor=logits_processor,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2975 in beam_search     │
│                                                                                                  │
│   2972 │   │   │   │   outputs, model_kwargs, is_encoder_decoder=self.config.is_encoder_decoder  │
│   2973 │   │   │   )                                                                             │
│   2974 │   │   │   if model_kwargs["past_key_values"] is not None:                               │
│ ❱ 2975 │   │   │   │   model_kwargs["past_key_values"] = se

In [ ]:
print(response)